In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bars
import warnings

# Suppress networkx warnings about path existence
warnings.filterwarnings('ignore', category=UserWarning)

def generate_topology(topology, n_nodes=1000, n_edges=5000):
    """Generate network topologies with reduced size for feasibility"""
    if topology == 'RN':
        G = nx.gnm_random_graph(n_nodes, n_edges)
    elif topology == 'BAN':
        m = max(1, n_edges // n_nodes)  # Ensure m ≥ 1
        G = nx.barabasi_albert_graph(n_nodes, m)
    elif topology == 'SLN':
        side = int(np.sqrt(n_nodes))
        G = nx.grid_2d_graph(side, side)
        G = nx.convert_node_labels_to_integers(G)
    elif topology == 'TLN':
        G = nx.triangular_lattice_graph(int(np.sqrt(n_nodes)), int(np.sqrt(n_nodes)))
        G = nx.convert_node_labels_to_integers(G)
    elif topology == 'HLN':
        G = nx.hexagonal_lattice_graph(int(np.sqrt(n_nodes)), int(np.sqrt(n_nodes)))
        G = nx.convert_node_labels_to_integers(G)
    else:
        raise ValueError(f"Unsupported topology: {topology}")

    # Ensure we have exactly n_edges
    while G.number_of_edges() < n_edges:
        u, v = np.random.choice(G.nodes(), 2, replace=False)
        if not G.has_edge(u, v):
            G.add_edge(u, v)
    return G

def purification(C1, C2):
    """Deutsch's purification protocol"""
    return (13 + 14*C1 + 14*C2 + 40*C1*C2) / (41 + 4*C1 + 4*C2 + 32*C1*C2)

def get_alternate_paths(G, u, v, main_path, max_path_length=5):
    """Find alternate paths between u and v excluding main path nodes"""
    forbidden = set(main_path) - {u, v}
    subgraph = G.copy()
    subgraph.remove_nodes_from(forbidden)

    paths = []
    try:
        # Limit path search with cutoff
        for path in nx.all_simple_paths(subgraph, u, v, cutoff=max_path_length):
            paths.append(len(path) - 1)  # Store path lengths
    except nx.NetworkXNoPath:
        pass

    # Include direct edge if exists
    if G.has_edge(u, v):
        paths.append(1)
    return paths

def simulate_strategy(G, pairs, delta, strategy='SPF'):
    """Simulate either SPF or SPL strategy"""
    concurrences = []

    for s, d, path in tqdm(pairs, desc=f"Processing {strategy}"):
        C_total = 1.0

        for i in range(len(path) - 1):
            u, v = path[i], path[i+1]
            path_lengths = get_alternate_paths(G, u, v, path)

            if not path_lengths:
                C_total = 0
                break

            # Sort paths according to strategy
            if strategy == 'SPF':
                path_lengths = sorted(path_lengths)
            else:  # SPL
                path_lengths = sorted(path_lengths, reverse=True)

            # Compute purified concurrence
            Cs = [max(0, 1 - l * delta) for l in path_lengths]
            current_C = Cs[0]
            for c in Cs[1:]:
                current_C = purification(current_C, c)

            C_total *= current_C

        concurrences.append(C_total)

    return np.mean(concurrences) if concurrences else 0.0

def main_simulation(topologies=['RN', 'BAN', 'SLN'], delta=0.02, l0_values=range(1, 4), num_samples=50):
    """Main simulation function with progress tracking"""
    results = {topo: {'SPF': {}, 'SPL': {}} for topo in topologies}

    for topo in topologies:
        print(f"\n=== Processing {topo} topology ===")
        G = generate_topology(topo)
        print(f"Generated {topo} with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

        # Precompute shortest paths for efficiency
        print("Precomputing shortest paths...")
        path_dict = dict(nx.all_pairs_shortest_path_length(G))

        for l0 in l0_values:
            print(f"\nProcessing l0 = {l0}")
            pairs = []

            # Collect source-destination pairs
            nodes = list(G.nodes())
            np.random.shuffle(nodes)
            for s in nodes:
                if len(pairs) >= num_samples:
                    break
                for d in nodes:
                    if s == d:
                        continue
                    try:
                        if path_dict[s][d] == l0:
                            path = nx.shortest_path(G, s, d)
                            pairs.append((s, d, path))
                            if len(pairs) >= num_samples:
                                break
                    except KeyError:
                        pass

            if not pairs:
                print(f"No S-D pairs found for l0={l0}")
                results[topo]['SPF'][l0] = 0.0
                results[topo]['SPL'][l0] = 0.0
                continue

            print(f"Found {len(pairs)} S-D pairs for l0={l0}")

            # Run both strategies
            results[topo]['SPF'][l0] = simulate_strategy(G, pairs, delta, 'SPF')
            results[topo]['SPL'][l0] = simulate_strategy(G, pairs, delta, 'SPL')

    # Plot results
    plot_results(results, l0_values)

def plot_results(results, l0_values):
    """Visualize the simulation results"""
    plt.figure(figsize=(12, 6))

    for topo in results.keys():
        spf_values = [results[topo]['SPF'].get(l0, 0) for l0 in l0_values]
        spl_values = [results[topo]['SPL'].get(l0, 0) for l0 in l0_values]

        plt.plot(l0_values, spf_values, 'o-', label=f'{topo} SPF')
        plt.plot(l0_values, spl_values, 'x--', label=f'{topo} SPL')

    plt.xlabel('Shortest Path Length (l0)')
    plt.ylabel('Average Concurrence')
    plt.title('Localized Multipath Entanglement Purification')
    plt.legend()
    plt.grid(True)
    plt.show()

# Run the simulation with reduced parameters
if __name__ == "__main__":
    main_simulation(
        topologies=['RN', 'BAN', 'SLN'],  # Test with simpler topologies first
        l0_values=range(1, 4),            # Test shorter path lengths
        num_samples=50                    # Reduced sample size
    )